In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from hscpy import parameters

RUNS = 65_001
ABC_4D = True  # infer tau from abc (4d abc)

In [ ]:
# RANGES!!
"""
Up-to-date ranges Feb 13:
    s (i.e. old s/tau) : 0.01 to 0.4
    sigma (i.e. old sigma/tau) : 0.001 to 0.1
    mu: 0.1 to 30
    tau: 0.1 to 10
    N: 25'000 to 900'000
"""
cells = np.ones(RUNS, dtype=int) * 100_000
rng = np.random.default_rng(seed=42)

if ABC_4D:
    taus = rng.uniform(0.1, 10, RUNS)
else:
    taus = np.ones(RUNS, dtype=int)
mu0s = rng.uniform(0.1, 30, RUNS)
# eta
etas = rng.uniform(0.001, 0.4, RUNS)
# sigma
sigmas = rng.uniform(0.001, 0.1, RUNS)

assert (etas * taus).max() < 4
assert (sigmas * taus).max() < 1

In [ ]:
plt.hist([parameters.compute_m_background(tau) for tau in taus], label="mu background", bins=100)
plt.legend()
plt.show()

sns.pairplot(
    pd.DataFrame(
        [mu0s, taus, etas, sigmas, cells], index=["mu", "tau", "eta", "sigma", "N"]
    ).T
)
plt.show()

In [ ]:
with open("parameters.txt", "w") as f:
    for tau, mu, mean, std, n in np.column_stack((taus, mu0s, etas, sigmas, cells)):
        c = int(n)
        f.write(
            f"-c {c} -y 82 -r 1 --tau {tau} --mu0 {mu} --mu-exp {parameters.compute_m_exp(c)} --mu-division 1.2 --mu-background {parameters.compute_m_background(tau)} --mean-std {parameters.compute_s_per_division_from_s_per_year(mean, tau)} {parameters.compute_std_per_division_from_std_per_year(std, tau)}\n"
        )

In [ ]:
%%bash
head parameters.txt

In [ ]:
%%bash --out runs
wc -l parameters.txt

In [ ]:
runs = int(runs.split(" ")[0])
assert RUNS == runs, f"{RUNS} vs {runs}"